In [29]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.preprocessing import StandardScaler

MERGED_DATA_DIR = "../../data/merged data"
OUTPUT_DIR = "../../output/assumption"

# Load the dataset
tnp_20 = pd.read_csv(os.path.join(MERGED_DATA_DIR, "2020", "merged_tnp_data.csv"))
tnp_19 = pd.read_csv(os.path.join(MERGED_DATA_DIR, "2019", "merged_tnp_data.csv"))

In [30]:
# 添加年份指示变量
tnp_20["is_2020"] = 1
tnp_19["is_2020"] = 0

scaler = StandardScaler()
cols_to_scale = ["daily_bus_rides", "rides"]
tnp_20[cols_to_scale] = scaler.fit_transform(tnp_20[cols_to_scale])
tnp_19[cols_to_scale] = scaler.fit_transform(tnp_19[cols_to_scale])

# 拼接数据
merged_df = pd.concat([tnp_20, tnp_19], ignore_index=True)
merged_df["price"] = merged_df["fare"] + merged_df["additional_charges"]
# merged_df['price'] = np.log1p(merged_df['price'])

# 确保日期变量是 datetime 类型（后续 RDiT 会用到）
merged_df["trip_start_date"] = pd.to_datetime(merged_df["trip_start_date"])

# 创建工作日虚拟变量，drop_first=True 是为了避免虚拟变量陷阱
day_dummies = pd.get_dummies(merged_df['day_of_week'], prefix='dow', drop_first=True)

# 合并到原始数据中
merged_df = pd.concat([merged_df, day_dummies], axis=1)

# 创建地区虚拟变量，drop_first=True 是为了避免虚拟变量陷阱
area_dummies = pd.get_dummies(merged_df['area_type'], prefix='at', drop_first=True)

# 合并到原始数据中
merged_df = pd.concat([merged_df, area_dummies], axis=1)

In [31]:
print("数据总行数：", len(merged_df))
print("是否有NaN：", merged_df["price"].isna().sum())
print("时间列范围：", merged_df["trip_start_date"].min(), merged_df["trip_start_date"].max())

数据总行数： 3187906
是否有NaN： 0
时间列范围： 2018-12-10 00:00:00 2020-02-05 00:00:00


In [ ]:
trip_controls = [
    "trip_seconds", "trip_miles", "trip_during_peak"
]

weather_controls = [
    "Avg_Temp_C", "Precipitation_mm",
    "Snowfall_mm", "Avg_Wind_Speed_mps",
]

substitutes_controls = [
    # "total_rides", "taxi",
    "rides", "daily_bus_rides", "taxi",
]

day_of_week_controls = ["dow_1", "dow_2", "dow_3", "dow_4"]

area_type_controls = ["at_1", "at_2"]

control_vars = (
    trip_controls
    + weather_controls
    + substitutes_controls
    + day_of_week_controls
    + area_type_controls
)

merged_df_0 = merged_df[merged_df["Cluster"] == 0]
merged_df_1 = merged_df[merged_df["Cluster"] == 1]
merged_df_2 = merged_df[merged_df["Cluster"] == 2]

# Single Bandwidth Result

In [32]:
def estimate_dif_in_rdit(
    df: pd.DataFrame,
    outcome: str,
    time_var: str,
    treat_year_var: str,
    cutoff_date,
    placebo_cutoff_date="2019-01-07",
    heterogeneity_vars: list = None,
    covariates: list = None,
    trend_order: int = 1,
    bandwidth: int = 29
):
    """
    估计Dif-in-RDiT模型，支持多个异质性变量（连续或虚拟变量）

    数学模型：
    Y_{it} = β0 + β1·Post_t + β2·TreatYear_i + β3·(Post_t·TreatYear_i)
           + f(TFC_t) + f(TFC_t)·Post_t + f(TFC_t)·TreatYear_i + f(TFC_t)·Post_t·TreatYear_i
           + ∑ δ_k·H_{ik} + ∑ θ_k·(Post_t·H_{ik}) + γ^T X + u_{it}
    """

    import pandas as pd
    import numpy as np
    import statsmodels.formula.api as smf

    df = df.copy()
    df[time_var] = pd.to_datetime(df[time_var])
    cutoff_date = pd.to_datetime(cutoff_date)
    placebo_cutoff_date = pd.to_datetime(placebo_cutoff_date)

    df["cutoff_for_row"] = df[treat_year_var].apply(
        lambda x: cutoff_date if x == 1 else placebo_cutoff_date
    )
    df["days_from_cutoff"] = (df[time_var] - df["cutoff_for_row"]).dt.days
    df = df[df["days_from_cutoff"].between(-bandwidth, bandwidth)]

    df["post_cutoff"] = (df["days_from_cutoff"] >= 0).astype(int)
    df["post_treat"] = df["post_cutoff"] * df[treat_year_var]

    trend_terms = []
    for i in range(1, trend_order + 1):
        base = f"days_from_cutoff_pow{i}"
        df[base] = df["days_from_cutoff"] ** i

        post = f"{base}_x_post"
        treat = f"{base}_x_treat"
        post_treat = f"{base}_x_post_treat"

        df[post] = df[base] * df["post_cutoff"]
        df[treat] = df[base] * df[treat_year_var]
        df[post_treat] = df[base] * df["post_cutoff"] * df[treat_year_var]

        trend_terms += [base, post, treat, post_treat]

    rhs = ["post_cutoff", treat_year_var, "post_treat"] + trend_terms

    if covariates:
        rhs += covariates

    interaction_terms = []
    if heterogeneity_vars:
        for var in heterogeneity_vars:
            rhs.append(var)  # H_k
            inter = f"post_cutoff:{var}"  # Post_t * H_k
            rhs.append(inter)
            interaction_terms.append(inter)

    formula = f"{outcome} ~ " + " + ".join(rhs)
    model = smf.ols(formula=formula, data=df).fit(cov_type='HC3')

    summary_df = model.summary2().tables[1].copy()
    summary_df.columns = summary_df.columns.astype(str)
    summary_df = summary_df.rename(columns={
        "Coef.": "coef",
        "Std.Err.": "std_err",
        "P>|t|": "p_value",
        "[0.025": "ci_lower",
        "0.975]": "ci_upper"
    })
    summary_df["variable"] = summary_df.index
    summary_df.reset_index(drop=True, inplace=True)
    print(model.rsquared)

    return summary_df

In [33]:
def export_grouped_rdit_to_tex(
    summary_df: pd.DataFrame,
    file_path: str = "grouped_rdit_table.tex",
    sig_levels: list = [0.1, 0.05, 0.01],
    caption: str = "Estimated effects of the congestion tax by income group.",
    label: str = "tab:rdit_by_income",
    group_labels: dict = None,
    group_obs: dict = None  # ✅ 新增参数：每组观测数
):
    import os
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    def get_stars(p):
        if p < sig_levels[2]: return '***'
        elif p < sig_levels[1]: return '**'
        elif p < sig_levels[0]: return '*'
        return ''
    
    def get_stars_from_dict(d):
        if 'p_value' in d:
            p = d['p_value']
        else:
            if 'coef' in d and 'stderr' in d and d['stderr'] != 0:
                import scipy.stats as stats
                t_stat = d['coef'] / d['stderr']
                p = 2 * (1 - stats.norm.cdf(abs(t_stat)))
            else:
                p = 1.0
        return get_stars(p)

    df = summary_df[summary_df["variable"] == "post_treat"].copy()
    groups = df["group"].tolist()

    columns = [group_labels.get(g, g) for g in groups] if group_labels else groups

    coef_row = [
        f"{row['coef']:.3f}{get_stars(row['P>|z|'])}"
        for _, row in df.iterrows()
    ]
    stderr_row = [
        f"({row['std_err']:.3f})"
        for _, row in df.iterrows()
    ]

    tex_lines = [
        "\\begin{table}[H]\\centering",
        f"\\caption{{{caption}}}",
        f"\\label{{{label}}}",
        "\\footnotesize",
        f"\\begin{{tabular}}{{l{'c' * len(columns)}}}",
        "\\toprule",
        " & " + " & ".join(columns) + " \\\\",
        "\\midrule",
        "Post $\\times$ TreatYear & " + " & ".join(coef_row) + " \\\\",
        "Std. Error & " + " & ".join(stderr_row) + " \\\\",
    ]

    # ✅ 添加观测数行（如果有提供）
    if group_obs:
        obs_row = ["Observations"] + [str(group_obs.get(g, "–")) for g in groups]
        tex_lines.append("\\midrule")
        tex_lines.append(" & ".join(obs_row) + " \\\\")

    tex_lines += [
        "\\bottomrule",
        "\\end{tabular}",
        "\\vspace{0.5em}",
        "\\begin{minipage}{0.95\\textwidth}\\footnotesize\\textit{Notes:} Robust standard errors in parentheses. * $p < 0.1$, ** $p < 0.05$, *** $p < 0.01$.\\end{minipage}",
        "\\end{table}"
    ]

    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(tex_lines))

    print(f"LaTeX table saved to: {file_path}")


In [ ]:
# Step 1: 拟合模型，仅返回关键系数（post_treat + 异质性）
summary_pooled = estimate_dif_in_rdit(
    df=merged_df,
    outcome="price",
    time_var="trip_start_date",
    treat_year_var="is_2020",
    cutoff_date="2020-01-06",
    placebo_cutoff_date="2019-01-07",
    covariates=control_vars,
    trend_order=1,
    bandwidth=15
)

for col in control_vars:
    print(f"{col}: {merged_df[col].nunique()} unique values")

summary_low = estimate_dif_in_rdit(
    df=merged_df_0,
    outcome="price",
    time_var="trip_start_date",
    treat_year_var="is_2020",
    cutoff_date="2020-01-06",
    placebo_cutoff_date="2019-01-07",
    covariates=control_vars,
    trend_order=1,
    bandwidth=15
)

summary_high = estimate_dif_in_rdit(
    df=merged_df_1,
    outcome="price",
    time_var="trip_start_date",
    treat_year_var="is_2020",
    cutoff_date="2020-01-06",
    placebo_cutoff_date="2019-01-07",
    covariates=control_vars,
    trend_order=1,
    bandwidth=15
)

summary_mid = estimate_dif_in_rdit(
    df=merged_df_2,
    outcome="price",
    time_var="trip_start_date",
    treat_year_var="is_2020",
    cutoff_date="2020-01-06",
    placebo_cutoff_date="2019-01-07",
    covariates=control_vars,
    trend_order=1,
    bandwidth=15
)

# Step 2: 导出表格
# 提取主系数并合并
rows = []
for label, df in [("Pooled", summary_pooled), ("Low Income", summary_low), ("Mid Income", summary_mid), ("High Income", summary_high)]:
    row = df[df["variable"] == "post_treat"].copy()
    row["group"] = label
    rows.append(row)

summary_df = pd.concat(rows)

from pandas import Timestamp

def count_bandwidth_sample(df, time_var, treat_year_var, cutoff_date, placebo_cutoff_date, bandwidth):
    df = df.copy()
    df[time_var] = pd.to_datetime(df[time_var])
    df["cutoff_for_row"] = df[treat_year_var].apply(
        lambda x: Timestamp(cutoff_date) if x == 1 else Timestamp(placebo_cutoff_date)
    )
    df["days_from_cutoff"] = (df[time_var] - df["cutoff_for_row"]).dt.days
    return df[df["days_from_cutoff"].between(-bandwidth, bandwidth)].shape[0]

GROUP_OBS = {
    "Pooled": count_bandwidth_sample(merged_df, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
    "Low Income": count_bandwidth_sample(merged_df_0, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
    "Mid Income": count_bandwidth_sample(merged_df_2, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
    "High Income": count_bandwidth_sample(merged_df_1, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
}

print(GROUP_OBS)

export_grouped_rdit_to_tex(
    summary_df=summary_df,
    file_path="../../output/regression/rdit_group_table.tex",
    caption="Estimated Treatment Effects of the Congestion Tax across Income Groups",
    label="tab:tax_by_income",
    group_obs=GROUP_OBS
)

0.7795886814191689
trip_seconds: 1457 unique values
trip_miles: 2209763 unique values
trip_during_peak: 2 unique values
Avg_Temp_C: 69 unique values
Precipitation_mm: 23 unique values
Snowfall_mm: 14 unique values
Avg_Wind_Speed_mps: 41 unique values
rides: 5649 unique values
daily_bus_rides: 7158 unique values
taxi: 55 unique values
dow_1: 2 unique values
dow_2: 2 unique values
dow_3: 2 unique values
dow_4: 2 unique values
at_1: 2 unique values
at_2: 2 unique values
0.8139738498096765
0.7688939921582473
0.7951670006904098
{'Pooled': 1385310, 'Low Income': 53645, 'Mid Income': 410354, 'High Income': 921311}
LaTeX table saved to: ../../output/regression/rdit_group_table.tex


# Controller Result

In [35]:
def export_controls_table_to_tex(
    summary_df: pd.DataFrame,
    file_path: str = "controls_table.tex",
    sig_levels: list = [0.1, 0.05, 0.01],
    caption: str = "Estimated coefficients on control variables across income groups.",
    label: str = "tab:controls_by_income",
    group_labels: dict = None,
    controls_list: list = None,
    control_labels: dict = None,
    group_obs: dict = None  # ✅ 新增参数：每组观测数
):
    import os
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    def get_stars(p):
        if p < sig_levels[2]: return '***'
        elif p < sig_levels[1]: return '**'
        elif p < sig_levels[0]: return '*'
        return ''

    if controls_list is None:
        controls_list = summary_df['variable'].tolist()

    df = summary_df[summary_df["variable"].isin(controls_list)].copy()
    grouped = df.groupby("group")
    control_vars = sorted(df['variable'].unique().tolist())

    columns = []
    rows_coef = {var: [] for var in control_vars}
    rows_se = {var: [] for var in control_vars}

    group_keys = []
    for group, subdf in grouped:
        col_label = group_labels.get(group, group) if group_labels else group
        columns.append(col_label)
        group_keys.append(group)  # 原始 group 名称
        subdf = subdf.set_index("variable")
        for var in control_vars:
            if var in subdf.index:
                coef = subdf.loc[var, 'coef']
                se = subdf.loc[var, 'std_err']
                pval = subdf.loc[var, 'P>|z|']
                stars = get_stars(pval)
                rows_coef[var].append(f"{coef:.3f}{stars}")
                rows_se[var].append(f"({se:.3f})")
            else:
                rows_coef[var].append("–")
                rows_se[var].append(" ")

    tex_lines = [
        "\\begin{table}[H]\\centering",
        f"\\caption{{{caption}}}",
        f"\\label{{{label}}}",
        "\\footnotesize",
        f"\\begin{{tabular}}{{l{'c' * len(columns)}}}",
        "\\toprule",
        " & " + " & ".join(columns) + " \\\\",
        "\\midrule"
    ]

    for var in control_vars:
        display_name = control_labels.get(var, var).replace("_", "\\_") if control_labels else var.replace("_", "\\_")
        tex_lines.append(display_name + " & " + " & ".join(rows_coef[var]) + " \\\\")
        tex_lines.append(" & " + " & ".join(rows_se[var]) + " \\\\")

    # ✅ 添加观测数行
    if group_obs:
        obs_row = ["Observations"]
        for g in group_keys:
            obs_row.append(str(group_obs.get(g, "–")))
        tex_lines.append("\\midrule")
        tex_lines.append(" & ".join(obs_row) + " \\\\")

    tex_lines += [
        "\\bottomrule",
        "\\end{tabular}",
        "\\vspace{0.5em}",
        "\\begin{minipage}{0.95\\textwidth}\\footnotesize\\textit{Notes:} Robust standard errors in parentheses. * $p < 0.1$, ** $p < 0.05$, *** $p < 0.01$.\\end{minipage}",
        "\\end{table}"
    ]

    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(tex_lines))

    print(f"LaTeX control variable table saved to: {file_path}")

In [36]:
# ✅ 设置 group 标签与顺序
group_labels = {
    "Overall": "Pooled",
    "Low": "Low Income",
    "Mid": "Middle Income",
    "High": "High Income",
}
group_order = list(group_labels.keys())

# ✅ 拼接 summary 并排序
summary_df = pd.concat([
    summary_pooled.assign(group="Overall"),
    summary_low.assign(group="Low"),
    summary_high.assign(group="High"),
    summary_mid.assign(group="Mid"),
])

summary_df["group"] = pd.Categorical(summary_df["group"], categories=group_order, ordered=True)
summary_df = summary_df.sort_values("group")

GROUP_OBS = {
    "Overall": count_bandwidth_sample(merged_df, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
    "Low": count_bandwidth_sample(merged_df_0, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
    "Mid": count_bandwidth_sample(merged_df_2, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
    "High": count_bandwidth_sample(merged_df_1, "trip_start_date", "is_2020", "2020-01-06", "2019-01-07", 15),
}

# ✅ 控制变量 label
control_labels = {
    "trip_seconds": "Trip Duration (s)",
    "trip_miles": "Trip Distance (miles)",
    "trip_during_peak": "Peak Hour Trip",
    "taxi": "Taxi Trips",
    # "total_rides": "Public Transportation Rides",
    "rides": "L Rail",
    "daily_bus_rides": "Bus",
    "Avg_Temp_C": "Temperature (°C)",
    "Precipitation_mm": "Precipitation (mm)",
    "Snowfall_mm": "Snowfall (mm)",
    "Avg_Wind_Speed_mps": "Wind Speed (m/s)",
    "dow_1": "Tuesday",
    "dow_2": "Wednesday",
    "dow_3": "Thursday",
    "dow_4": "Friday",
    "at_1": "Loop",
    "at_2": "Tourist/Transit Area",
}

# ✅ 导出表格
export_controls_table_to_tex(
    summary_df=summary_df,
    file_path="../../output/regression/rdit_controls_table.tex",
    caption="Estimated Coefficients on Control Variables across Income Groups",
    label="tab:controls_by_income",
    group_labels=group_labels,
    group_obs=GROUP_OBS,
    controls_list=control_vars,
    control_labels=control_labels
)

LaTeX control variable table saved to: ../../output/regression/rdit_controls_table.tex


/var/folders/_x/43n5hx2d6hqf1v6v65qbln8c0000gn/T/ipykernel_14770/2682482435.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby("group")
